In [1]:
import sys, time, random, math, cv2
import numpy as np
from matplotlib import pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

sys.path.insert(0, '../src')
from robot import Robot

In [2]:
class GoToGoalFuzzy():
    def __init__(self):
        ## Antecedents ##
        # Distance in meters
        distance = ctrl.Antecedent(np.arange(0, 20, 0.5), 'distance')

        # Angular distance
        angular_distance = ctrl.Antecedent(np.arange(0, 360, 1), 'angular distance') 

        ## Consequents ##
        # Velocity on the right wheel
        right_wheel = ctrl.Consequent(np.arange(0, 4, 0.5), 'right wheel speed')
        # Velocity on the left wheel
        left_wheel = ctrl.Consequent(np.arange(0, 4, 0.5), 'left wheel speed') 

        ## Memberships distance functions ##
        distance['none'] = fuzz.trimf(distance.universe, [0, 0, 0.15])
        distance['small'] = fuzz.trimf(distance.universe, [0, 0.2, 0.8])
        distance['medium'] = fuzz.trapmf(distance.universe, [0.7, 1.5, 2, 3])
        distance['big'] = fuzz.trapmf(distance.universe, [2.5, 5, 20, 20])

        ## Angular ##
        angular_distance['small from left'] = fuzz.trimf(angular_distance.universe, [0, 0, 15]) # diferença angular pequena pela esquerda
        angular_distance['smaller from left'] = fuzz.trapmf(angular_distance.universe, [0, 0, 179, 180])
        angular_distance['smaller from right'] = fuzz.trapmf(angular_distance.universe, [180, 181, 359, 360])
        angular_distance['small from right'] = fuzz.trimf(angular_distance.universe, [345, 360, 360])

        ## Right wheel ##
        right_wheel['zero'] = fuzz.trimf(right_wheel.universe, [0, 0, 0])
        right_wheel['low'] = fuzz.trimf(right_wheel.universe, [0, 0, 1.5])
        right_wheel['high'] = fuzz.trapmf(right_wheel.universe, [1, 2, 4, 4])

        ## Left wheel ##
        left_wheel['zero'] = fuzz.trimf(left_wheel.universe, [0, 0, 0])
        left_wheel['low'] = fuzz.trimf(left_wheel.universe, [0, 0, 1.5])
        left_wheel['high'] = fuzz.trapmf(left_wheel.universe, [1, 2, 4, 4])

        # Rules
        rules = []

        # Stop
        rules.append(ctrl.Rule(distance['none'], [right_wheel['zero'],  left_wheel['zero']]))

        # Turn
        rules.append(ctrl.Rule((distance['big'] | distance['medium']) & angular_distance['smaller from left'] & (~angular_distance['small from left']), [right_wheel['high'], left_wheel['low']] ))

        rules.append(ctrl.Rule((distance['big'] | distance['medium']) & angular_distance['smaller from right'] & (~angular_distance['small from right']), [right_wheel['low'],  left_wheel['high']]))

        rules.append(ctrl.Rule(angular_distance['smaller from left'], [right_wheel['low'], left_wheel['zero']]))

        rules.append(ctrl.Rule(angular_distance['smaller from right'], [right_wheel['zero'], left_wheel['low']]))

        # Follow
        rules.append(ctrl.Rule(angular_distance['small from right'] | angular_distance['small from left'], [right_wheel['high'], left_wheel['high']]))

        rules.append(ctrl.Rule(distance['small'] & (angular_distance['small from right'] | angular_distance['small from left']), [right_wheel['low'], left_wheel['low']]))

         # Control and simulation
        self.goToGoal_ctrl = ctrl.ControlSystem(rules)
        self.goToGoal_sim = ctrl.ControlSystemSimulation(self.goToGoal_ctrl)

    def get_fuzzy_control(self, dist, ang_dist):
        # Stop
        if (dist <= 0.1):
            return [0, 0, ]
        else:
            self.goToGoal_sim.input['distance'] = dist
            self.goToGoal_sim.input['angular distance'] = ang_dist
            self.goToGoal_sim.compute()
            return [self.goToGoal_sim.output['right wheel speed'], self.goToGoal_sim.output['left wheel speed'],]

def cicle(ang):
    return (ang + 2 * math.pi) % (2*math.pi)

def get_distance_from_goal(a,b):
    return np.linalg.norm(a-b)

def get_angular_distance(v1,v2):

    v1_aux = v1 / np.linalg.norm(v1)
    v2_aux = v2 / np.linalg.norm(v2)

    theta = np.arccos(np.clip(np.dot(v1_aux, v2_aux), -1.0, 1.0))
    rot_theta = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    theta = theta*180.0/math.pi
    theta = (360.0+theta)%360.0

    if abs(np.dot(np.dot(rot_theta,v2_aux),v1_aux)-1.0) > 1e-2:
        theta = 360.0 - theta

    return theta

def unit_vector(vector):
    return vector / np.linalg.norm(vector)


In [3]:
ls = ctrl.Antecedent(np.arange(0,10.1,0.1), 'left sensor')
fs = ctrl.Antecedent(np.arange(0,10.1,0.1), 'front sensors')
rs = ctrl.Antecedent(np.arange(0,10.1,0.1), 'right sensor')

ls['near'] = fuzz.trimf(ls.universe, [0, 0, 3])
ls['far'] = fuzz.trapmf(ls.universe, [0, 3, 10, 10])

fs['near'] = fuzz.trimf(fs.universe, [0, 0, 3])
fs['far'] = fuzz.trapmf(ls.universe, [0, 3, 10, 10])

rs['near'] = fuzz.trimf(rs.universe, [0, 0, 3])
rs['far'] = fuzz.trapmf(ls.universe, [0, 3, 10, 10])

f_vel = ctrl.Consequent(np.arange(0, 11, 1), 'velocity')
f_angle = ctrl.Consequent(np.arange(-2,2,0.1), 'angle')

f_vel['no movement'] = fuzz.trimf(f_vel.universe, [0, 0, 3])
f_vel['slow'] = fuzz.trimf(f_vel.universe, [0, 3, 6])
f_vel['fast'] = fuzz.trapmf(f_vel.universe, [3, 6, 10, 10])

f_angle['left'] = fuzz.trimf(f_angle.universe, [-2, -2, 0])
f_angle['forward'] = fuzz.trimf(f_angle.universe, [-0.2, 0, 0.2])
f_angle['right'] = fuzz.trimf(f_angle.universe, [0, 2, 2])

rules = []
rules.append(ctrl.Rule(ls['near'] & fs['near'] & rs['near'], [f_vel['no movement'], f_angle['left']]))
rules.append(ctrl.Rule(ls['far'] & fs['near'] & rs['far'], [f_vel['slow'], f_angle['forward']]))
rules.append(ctrl.Rule(ls['near'] & fs['far'] & rs['near'], [f_vel['slow'], f_angle['forward']]))
rules.append(ctrl.Rule(ls['far'] & fs['far'] & rs['far'], [f_vel['fast'], f_angle['forward']]))

rules.append(ctrl.Rule(ls['near'] & fs['near'] & rs['far'], [f_vel['slow'], f_angle['left']]))
rules.append(ctrl.Rule(ls['far'] & fs['near'] & rs['near'], [f_vel['slow'], f_angle['right']]))

rules.append(ctrl.Rule(ls['near'] & fs['far'] & rs['far'], [f_vel['fast'], f_angle['forward']]))
rules.append(ctrl.Rule(ls['far'] & fs['far'] & rs['near'], [f_vel['fast'], f_angle['forward']]))

def avoid_obstacles(sensors):
    avoid_ctrl = ctrl.ControlSystem(rules)
    avoid = ctrl.ControlSystemSimulation(avoid_ctrl)
    
    avoid.input['left sensor'] = sensors[2]
    avoid.input['front sensors'] = min(sensors[3], sensors[4])
    avoid.input['right sensor'] = sensors[5]
    
    avoid.compute()
    
    vel = avoid.output['velocity']
    angle = avoid.output['angle']
    
    return [(1 - max(0, angle)) * vel, (1 + min(0, angle)) * vel]
    

In [4]:
robot = Robot("")

goToGoalFuzzy = GoToGoalFuzzy()

position = robot.get_current_position()
orientation = robot.get_current_orientation()

goal=[np.array([13, 0]), np.array([-13,0])]
idx = 1

score = 0

while robot.get_connection_status() != -1:
    
    current_position = np.array(robot.get_current_position()[:-1])
    current_angle = cicle(robot.get_current_orientation()[2])

    distance = get_distance_from_goal(current_position, goal[idx])
    angle_distance = get_angular_distance(goal[idx] - current_position, np.array([math.cos(current_angle),math.sin(current_angle)])) # graus

    vel = goToGoalFuzzy.get_fuzzy_control(distance, angle_distance)
    
    us_distances = robot.read_ultrassonic_sensors()

    vel_ = avoid_obstacles(us_distances)
    
    flag = False
    for k in us_distances[:8]:
        flag = flag or (k < 0.3)
    
    if (flag):
        robot.set_left_velocity(vel_[0] * 4)
        robot.set_right_velocity(vel_[1] * 4)
    else:
        robot.set_left_velocity(vel[1] * 4)
        robot.set_right_velocity(vel[0] * 4)

    if get_distance_from_goal(current_position, goal[idx]) < 0.3:
        idx = (idx + 1) % 2
        score += 1
        


Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser not connected.
 Left motor connected.
 Right motor connected.
 Robot connected.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pedro/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4ac3b9d2a93b>", line 29, in <module>
    vel_ = avoid_obstacles(us_distances)
  File "<ipython-input-3-1fb028a1812b>", line 45, in avoid_obstacles
    avoid.compute()
  File "/home/pedro/anaconda3/lib/python3.7/site-packages/skfuzzy/control/controlsystem.py", line 361, in compute
    for rule in self.ctrl.rules:
  File "/home/pedro/anaconda3/lib/python3.7/site-packages/skfuzzy/control/controlsystem.py", line 729, in __iter__
    self._cache = list(self._process_rules(self.all_rules[:]))
  File "/home/pedro/anaconda3/lib/python3.7/site-packages/skfuzzy/control/controlsystem.py", line 766, in _process_rules
    self.calced_graph = nx.compose(self.calced_graph, rule.graph)
  File "/home/pedro/anaconda3/lib/python3.7/site-packages/skfuzzy/control/rule.py", line 30

KeyboardInterrupt: 